In [1]:
## fill nas with median
## very rude transformation of dates
## test/train split
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
# from fancyimpute import *

%matplotlib inline

In [2]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

def batch_save(train_x, train_y, valid_x, valid_y, test, postfix):
    train_x.reset_index().to_feather("tmp/train_x_{}".format(postfix))
    train_y.reset_index().to_feather("tmp/train_y_{}".format(postfix))
    valid_x.reset_index().to_feather("tmp/valid_x_{}".format(postfix))
    valid_y.reset_index().to_feather("tmp/valid_y_{}".format(postfix))
    test.reset_index().to_feather("tmp/test_{}".format(postfix))
    
def batch_load(postfix):
    train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
    train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
    valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
    valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
    return train_x, train_y, valid_x, valid_y

def my_roc(y_true, y_prob):
    if isinstance(y_true,pd.core.series.Series):
        y_true = np.array(y_true.tolist())
    if isinstance(y_true,list):
        y_true = np.array(y_true)
    sort_index = np.argsort(y_prob)[::-1]
    y_prob = y_prob[sort_index]
    y_true = y_true[sort_index]
    num_p = y_true.sum()
    num_n = len(y_true) - num_p
    fp = 0
    tp = 0
    fps = []
    tps = []
    prob_prev = -99
    i = 0
    while i < len(y_true):
        if y_prob[i]!=prob_prev:
            fps.append(fp/num_n)
            tps.append(tp/num_p)
            prob_prev=y_prob[i]
        if y_true[i]==1:
            tp+=1
        else:
            fp+=1
        i+=1
    fps.append(fp/num_n)
    tps.append(tp/num_p)
    return np.array(fps), np.array(tps)

def my_score3(predictions, xtrain): ##Adapted from SKlearn, conservative (actual should be higher)
    ground_truth = y_true = xtrain.get_label()
    fpr,tpr = my_roc(ground_truth, predictions)
#     plt.scatter(fpr, tpr)
#     plt.show()
    tpr1 = tpr[(fpr>=0.001).argmax()-1]
    tpr2 = tpr[(fpr>=0.005).argmax()-1] 
    tpr3 = tpr[(fpr>=0.01).argmax()-1]
    return 'score', 0.4 * tpr1 + 0.3 * tpr2 + 0.3* tpr3

def norm_standardize(df, start=0):
    for col in df.columns[start:]:
#         avg = df[col].mean()
#         std = df[col].std(ddof=0)
#         if std != 0:
#             df[col] = (df[col]-avg)/std
#         else:
#             print(col)
        a = df[col]
        z = a
        z[~np.isnan(a)] = zscore(a[~np.isnan(a)])
        df[col] = z
            
def norm_maxmin(df, start=0):
    for col in df.columns[start:]:
        df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())

In [3]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

**** Load the training data and test data ****

In [4]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)

test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype)

In [5]:
df_missing_ratio = load_obj('df_missing_ratio')
# display_all(df_missing_ratio)

selected_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']<0.1].index.tolist()]
all_nan_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']>0.9].index.tolist()]

# # use the columns with no or few missing values
# data = data.drop(all_nan_cols, axis=1)
# test = test.drop(all_nan_cols, axis=1)

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,2018-01-20,0,0,1,1,100809.0,1,3,2,...,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,2018-01-05,1,1,0,0,100808.0,1,5,2,...,39.0,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,2018-01-09,0,0,0,0,100803.0,0,6,1,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
3,0df905aa187938d56a9b0816b13f54ac1f87d658a33cd0...,2018-01-18,1,1,1,1,100808.0,0,3,2,...,301.0,301.0,302.0,122.0,301.0,208.0,208.0,207.0,89.0,207.0
4,15f532f979c4f092bbbe28e5409c8c3b8454ece8f1ab3e...,2018-01-09,1,1,1,1,100809.0,0,2,2,...,301.0,302.0,302.0,118.0,302.0,251.0,251.0,251.0,101.0,251.0
5,614728f2aba299d20ddffca7e1ebc38ae8f239663562a9...,2018-02-01,1,0,2,1,NaN,1,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3dc22d969895bc4afdaa5266d1586ae1a360adfcc638b5...,2018-02-03,0,1,0,1,100810.0,1,4,2,...,302.0,307.0,322.0,11.0,234.0,193.0,193.0,193.0,7.0,148.0
7,b6cba0173d6a894f443578b5d82c00444dfe24c5e1705d...,2018-01-15,0,0,0,0,100809.0,1,2,1,...,2.0,5.0,7.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
8,84d010a48c747947588589473e2ebdc59db3d348849c5b...,2018-01-10,1,1,1,1,100804.0,0,5,1,...,302.0,301.0,302.0,31.0,302.0,249.0,249.0,250.0,22.0,250.0
9,bc00fec7ff8b3e4f8bd826048da109e7fe40fcc604ba28...,2018-01-27,1,0,1,1,NaN,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**** Perform one hot encoding for the columns with no more than 10 unique values ****

In [11]:
# one hot encoding for the columns with no more than 10 unique values
for col in data.columns[3:]:
    data_unique = data[col].unique()
    test_unique = test[col].unique()
    if data_unique[~np.isnan(data_unique)].min() == test_unique[~np.isnan(test_unique)].min() and \
    data_unique[~np.isnan(data_unique)].max() == test_unique[~np.isnan(test_unique)].max() and \
    data_unique.shape[0] == test_unique.shape[0] and \
    data_unique.shape[0] <= 10:
        data[col].fillna(-1.0)
        test[col].fillna(-1.0)
        for num in data[col].unique():
            new_col = '{}={}'.format(col, num)
            data[new_col] = data[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            test[new_col] = test[col].apply(lambda x: np.isnan(x) if np.isnan(num) else x==num)
            data.drop([col], axis=1)
            test.drop([col], axis=1)

**** save the file locally to save the time of preprocessing ****

In [12]:
save_obj(data, 'train_onehot')
save_obj(test, 'test_onehot')

In [40]:
# load the training set and test set
data = load_obj('train_onehot')
test = load_obj('test_onehot')

**** Perform normalization ****

In [ ]:
# normalization with maxmin
norm_maxmin(data, 3)
norm_maxmin(test, 2)

In [ ]:
# normalization with zscore
norm_standardize(data, 3)
norm_standardize(test, 2)

**** Sort the training data and remove the unlabeled data ****

In [41]:
# temporially ignore the rows without labels
data.sort_values('date',inplace=True)
unlabeled = data[data['label']==-1]
data = data[data['label']!=-1]

**** Create validation set and training set ****

In [ ]:
train = data.iloc[:len(data) * 9 // 10]
valid = data.iloc[len(data) * 9 // 10:]

In [ ]:
train_y = train[['label']]
train_x = train.iloc[:,3:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,3:]

In [ ]:
xgtrain = xgb.DMatrix(train_x.values, label=train_y.values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2')]

**** Train the model ****

In [ ]:
# set up the parameters
params = {'max_depth': 8, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['auc', "error"]
params["scale_pos_weight"] = 5
num_rounds = 15
early_stopping_rounds = 1000

# set up the random seed for testing
params["seed"] = 10

In [ ]:
%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist, feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

**** save or load the model ****

In [ ]:
# save the model
xgb_model.save_model('model_log/0011.model')
# dump model with feature map
xgb_model.dump_model('model_log/dumpraw0011.txt')

In [17]:
# load the model
xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/0011.model')  # load model

**** train the existing model on the validation set ****

In [ ]:
xgval = xgb.DMatrix(valid_x.values, label=valid_y.values)

In [ ]:
%time xgb_model = xgb.train(params, xgval, num_rounds, [(xgval, 'validation')], feval=my_score3, early_stopping_rounds=early_stopping_rounds, xgb_model = xgb_model)

**** predict on the testset ****

In [ ]:
# load test data
test_whole = pd.read_feather("tmp/test_native")

In [54]:
xgtest = xgb.DMatrix(test.iloc[:,2:].values)

# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)

res = pd.concat([test.id, pd.Series(list(preds), name='score')], axis=1)

res.to_csv("0011.csv", index=False)

In [58]:
# the positive ratio of the test data
print('Ratio of positive label in unlabeled data: {}%'.format((preds>0.5).sum()/preds.shape[0]*100))

Ratio of positive label in unlabeled data: 0.9431160864648503%


**** predict on the unlabeled training set ****

In [15]:
xgunlabeled = xgb.DMatrix(unlabeled.iloc[:,3:].values)

In [43]:
unlabeled.reset_index(drop=True, inplace=True)

In [44]:
pred_xgunlabeled = xgb_model.predict(xgunlabeled)
res = pd.concat([unlabeled.id, pd.Series(list(pred_xgunlabeled), name='score')], axis=1)
res.to_csv("Yabin_unlabeled0011.csv", index=False)

In [53]:
# the positive ratio of the unlabeled data
print('Ratio of positive label in unlabeled data: {}%'.format((pred_xgunlabeled>0.5).sum()/res.shape[0]*100))

Ratio of positive label in unlabeled data: 17.756613756613756%
